## 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace' # Path to our workspace
SCRIPTS_PATH = 'Tensorflow/scripts' # Where generate tf record script is
APIMODEL_PATH = 'Tensorflow/models' # tf objection detection library is downloaded here
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations' # label records and tf records
IMAGE_PATH = WORKSPACE_PATH + '/images' # labelled images
MODEL_PATH = WORKSPACE_PATH + '/models' # where trained model will be present
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models' # Where our downloaded pre-trained model is saved
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config' # tf object detection library has a config file containing information about Deep Learning Model to be used
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/' # All check-points are saved

## 1. Create Label Map

In [2]:
# Categories in which images are labelled are set here
labels = [{'name':'Mask','id':1},{'name':'NoMask','id':2}]

In [3]:
labels

[{'name': 'Mask', 'id': 1}, {'name': 'NoMask', 'id': 2}]

In [4]:
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 2. Create TF Records

In [5]:
#!pip install tensorflow-object-detection-api

Since I have Tensorflow==2.0, I made these changes to the following files:
1. ***object_detection\utils\label_map_util.py*** `tf.gfile` is replaced by `tf.io.gfile` (line 132)
2. ***generate_tfrecord.py*** in (line 61 & 62)

    `label_map = label_map_util.load_labelmap(args.labels_path)`
    
    `label_map_dict = label_map_util.get_label_map_dict(label_map)`
    
    is replaced by a single statement
    
    `label_map_dict = label_map_util.get_label_map_dict(args.labels_path)`

In [6]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record


In [7]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [8]:
# I cloned this repository using Git Application
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [9]:
!pip install wget

In [10]:
import wget
wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

100% [........................................................................] 20515344 / 20515344

'mv' is not recognized as an internal or external command,
operable program or batch file.
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


## 4. Copy Model Config to Training Folder 

In [11]:
 CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

`cd` is replaced by `copy`
Now what I am doing is copying pre-trained model to folder specified by `MODEL_PATH/my_ssd_mobnet`

In [16]:
!mkdir {'Tensorflow\workspace\models\\' + CUSTOM_MODEL_NAME}
!copy {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


The syntax of the command is incorrect.


Hence above step was just a try

## 5. Update Config For Transfer Learning